<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/PytorchTrainingRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural net pytorch

## Setup Device

In [31]:
import torch
#!pip install torch

device = torch.device('cpu')
if (torch.cuda.is_available()):
  device = torch.device('cuda:0')
  print(device, torch.cuda.get_device_name(0))

cuda:0 Tesla T4


## Design Neural Net

In [32]:
import torch

class MyNet(torch.nn.Module):
  def __init__(self):
    super(MyNet, self).__init__()
    # Init the element
    self.h1 = torch.nn.Linear(1,50)
    self.h2 = torch.nn.Linear(50,30)
    self.o = torch.nn.Linear(30,1)
  def forward(self, x):
    # Feedforward
    x = torch.tanh( self.h1(x) )# x*W +b
    x = torch.tanh( self.h2(x) )# x*W +b
    y = self.o(x) # x*W +b
    return y

# Setup
net = MyNet().to(device)
#x = torch.tensor([[2],[3],[4]], dtype=torch.float32, device=device)

# Prediction
#y_hat = net(x)
#print(y_hat)
#print(y_hat.detach())

# Produzione
#with torch.no_grad():
#  y_hat = net(x)
#  print(y_hat.cpu().numpy())

## Data Loader

In [42]:
# Data loader
import plotly.graph_objects as go
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

x = torch.linspace(0,10,1000)
y = torch.sin(x)  + 0.1*torch.randn(x.shape[0])

x = x.reshape(-1,1)
y = y.reshape(-1,1)

# Trainint test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42, shuffle=True)

fig = go.Figure()
fig.add_traces( go.Scatter( x=x.flatten(),y=y.flatten(), mode="markers"))
fig.show()

class MyDataset(Dataset):
    def __init__(self, x, y):
      self.x = x.type(torch.float32)
      self.y = x.type(torch.float32)
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      return self.x[idx], self.y[idx]
  
train_ds = MyDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=10, shuffle=True)

test_ds = MyDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=10, shuffle=True)

# Load training data
for xb,yb in train_dl:
  print("Training Data", xb.shape, yb.shape)
  break

# Load test data
for el in test_dl:
  print("Test Data", el)
  break

Training Data torch.Size([10, 1]) torch.Size([10, 1])
Test Data [tensor([[9.9600],
        [7.9680],
        [8.9389],
        [7.5375],
        [2.9930],
        [4.2543],
        [6.5265],
        [9.2492],
        [3.0931],
        [6.6466]]), tensor([[9.9600],
        [7.9680],
        [8.9389],
        [7.5375],
        [2.9930],
        [4.2543],
        [6.5265],
        [9.2492],
        [3.0931],
        [6.6466]])]


## Training Loop

In [43]:
def train(train_dl, test_dl, net, loss_func, optimizer, metric_func, epochs):

  test_best_metric = 100000
  for epoch in range(0,epochs):
    
    # ------- Train set ( Update weights)
    net.train()
    
    loss_avg = 0
    count_loss = 0
    for xb,yb in train_dl:
      
      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Send the loaded data to the divice
      xb = xb.to(device)
      yb = yb.to(device)
      
      # Feedforward --> input --> output
      yb_hat = net(xb) 

      # Backpropagation output --> input
      # Compute the loss and its gradients
      train_loss = loss_func(yb_hat, yb)
      train_loss.backward()

      # Adjust learning weights
      optimizer.step()

      loss_avg += train_loss.item()
      count_loss += 1.0
    
    # ------- Test set ( Which model I want to save)
    net.eval()

    test_metric = 0.0
    count_metric = 0.0
    for xb,yb in test_dl:
      
      # Send the loaded data to the device
      xb = xb.to(device)
      yb = yb.to(device)

      # Prediction
      with torch.no_grad():
        yb_hat = net(xb) 
        err = metric_func(yb, yb_hat)
        test_metric += err.item()
        count_metric += 1.0
    
    #print("[RMSE] Errore sul test set: ", test_err/count_test)
    if (test_metric < test_best_metric):
      test_best_metric = test_metric
      #print("Model Saved")
      # https://pytorch.org/tutorials/beginner/saving_loading_models.html
      net_scripted = torch.jit.script(net) # Export to TorchScript
      net_scripted.save('net.pt') # Save
    
    # Print
    print(f"[Epoch] {epoch}/{epochs}, [Train Loss]: {round( loss_avg/count_loss, 3)} [Test Metric] {round( test_metric/count_metric, 3)}" )

def metric_func(yb, yb_hat):
   rmse = torch.sqrt( torch.mean((yb - yb_hat) ** 2) )
   return rmse

net = MyNet().to(device)
loss_func = torch.nn.MSELoss(reduction="sum") 
#optimizer = torch.optim.SGD(net.parameters(), lr=1e-4)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
train(train_dl, test_dl, net, loss_func, optimizer, metric_func, epochs=20)


[Epoch] 0/20, [Train Loss]: 151.25 [Test Metric] 2.722
[Epoch] 1/20, [Train Loss]: 59.835 [Test Metric] 2.115
[Epoch] 2/20, [Train Loss]: 36.474 [Test Metric] 1.644
[Epoch] 3/20, [Train Loss]: 21.214 [Test Metric] 1.206
[Epoch] 4/20, [Train Loss]: 10.936 [Test Metric] 0.834
[Epoch] 5/20, [Train Loss]: 5.475 [Test Metric] 0.608
[Epoch] 6/20, [Train Loss]: 2.959 [Test Metric] 0.468
[Epoch] 7/20, [Train Loss]: 1.782 [Test Metric] 0.342
[Epoch] 8/20, [Train Loss]: 1.143 [Test Metric] 0.278
[Epoch] 9/20, [Train Loss]: 0.788 [Test Metric] 0.245
[Epoch] 10/20, [Train Loss]: 0.559 [Test Metric] 0.207
[Epoch] 11/20, [Train Loss]: 0.407 [Test Metric] 0.178
[Epoch] 12/20, [Train Loss]: 0.3 [Test Metric] 0.152
[Epoch] 13/20, [Train Loss]: 0.234 [Test Metric] 0.126
[Epoch] 14/20, [Train Loss]: 0.18 [Test Metric] 0.111
[Epoch] 15/20, [Train Loss]: 0.144 [Test Metric] 0.109
[Epoch] 16/20, [Train Loss]: 0.119 [Test Metric] 0.099
[Epoch] 17/20, [Train Loss]: 0.101 [Test Metric] 0.088
[Epoch] 18/20, [Tr

## Production

In [ ]:
import torch

device = torch.device('cuda:0')
#device = torch.device('cpu')

# Load datatset to predict
x = torch.linspace(0,10,1000)
y = torch.sin(x) + 0.1*torch.randn(x.shape[0])
x = x.reshape(-1,1)
y = y.reshape(-1,1)

x = x.to(device)

# Load model
model = torch.jit.load('net.pt')
model = model.to(device)
model.eval()

# Prediction
with torch.no_grad():
  y_hat = model(x).cpu()

# Plot
fig = go.Figure()
fig.add_traces( go.Scatter( x=x.cpu().flatten(),y=y.flatten(), mode="markers", name="real"))
fig.add_traces( go.Scatter( x=x.cpu().flatten(),y=y_hat.flatten(), mode="markers", name="predict"))
fig.show()


## Linear Regression

In [30]:
import plotly.graph_objects as go
import torch
from sklearn.linear_model import LinearRegression

x = torch.linspace(0,10,1000)
X = torch.stack( [x,torch.pow(x,2),torch.pow(x,3),torch.pow(x,4)],1)
y = torch.sin(x) + 0.1*torch.randn(x.shape[0])
y = y.reshape(-1,1)
#x = x.reshape(-1,1)

model = LinearRegression()
model.fit(X,y)
y_hat = model.predict(X)

fig = go.Figure()
fig.add_traces( go.Scatter( x=x.flatten(),y=y.flatten(), mode="markers", name="real"))
fig.add_traces( go.Scatter( x=x.flatten(),y=y_hat.flatten(), mode="markers", name="predict"))
fig.show()